# End-to-End

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/translation](https://github.com/huseinzol05/Malaya/tree/master/example/translation).
    
</div>

<div class="alert alert-info">

This module trained on both standard and local (included social media) language structures, so it is save to use for both.
    
</div>

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
%%time

import malaya
import logging

logging.basicConfig(level=logging.INFO)

/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CPU times: user 2.78 s, sys: 3.51 s, total: 6.29 s
Wall time: 2.25 s


/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


### List available HuggingFace models

In [3]:
malaya.translation.available_huggingface

{'mesolitica/translation-t5-tiny-standard-bahasa-cased': {'Size (MB)': 139,
  'Suggested length': 1536,
  'en-ms chrF2++': 65.91,
  'ms-en chrF2++': 61.3,
  'ind-ms chrF2++': 58.15,
  'jav-ms chrF2++': 49.33,
  'pasar ms-ms chrF2++': 58.46,
  'pasar ms-en chrF2++': 55.76,
  'manglish-ms chrF2++': 51.04,
  'manglish-en chrF2++': 52.2,
  'from lang': ['en', 'ms', 'ind', 'jav', 'bjn', 'manglish', 'pasar ms'],
  'to lang': ['en', 'ms']},
 'mesolitica/translation-t5-small-standard-bahasa-cased': {'Size (MB)': 242,
  'Suggested length': 1536,
  'en-ms chrF2++': 67.37,
  'ms-en chrF2++': 63.79,
  'ind-ms chrF2++': 58.09,
  'jav-ms chrF2++': 52.11,
  'pasar ms-ms chrF2++': 62.49,
  'pasar ms-en chrF2++': 60.77,
  'manglish-ms chrF2++': 52.84,
  'manglish-en chrF2++': 53.65,
  'from lang': ['en', 'ms', 'ind', 'jav', 'bjn', 'manglish', 'pasar ms'],
  'to lang': ['en', 'ms']},
 'mesolitica/translation-t5-base-standard-bahasa-cased': {'Size (MB)': 892,
  'Suggested length': 1536,
  'en-ms chrF2++'

In [4]:
print(malaya.translation.info)

1. tested on FLORES200 pair `dev` set, https://github.com/huseinzol05/malay-dataset/tree/master/translation/flores200-eval
2. tested on noisy test set, https://github.com/huseinzol05/malay-dataset/tree/master/translation/noisy-eval
3. check out NLLB 200 metrics from `malaya.translation.nllb_metrics`.
4. check out Google Translate metrics from `malaya.translation.google_translate_metrics`.


### Improvements of new model

1. able to translate `[en, ms, ind, jav, bjn, manglish, pasar ms, mandarin, pasar mandarin]` while old model only able to translate `[en, ms, pasar ms]`.
2. No longer required `from_lang` part of the prefix.
3. able to retain text structure as it is.

### Load Transformer models

```python
def huggingface(
    model: str = 'mesolitica/translation-t5-small-standard-bahasa-cased',
    force_check: bool = True,
    from_lang: List[str] = None,
    to_lang: List[str] = None,
    old_model: bool = False,
    **kwargs,
):
    """
    Load HuggingFace model to translate.

    Parameters
    ----------
    model: str, optional (default='mesolitica/translation-t5-small-standard-bahasa-cased')
        Check available models at `malaya.translation.available_huggingface()`.
    force_check: bool, optional (default=True)
        Force check model one of malaya model.
        Set to False if you have your own huggingface model.

    Returns
    -------
    result: malaya.torch_model.huggingface.Translation
    """
```

In [5]:
model = malaya.translation.huggingface(model = 'mesolitica/translation-nanot5-small-malaysian-cased')

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Translate

```python
def generate(self, strings: List[str], to_lang: str = 'ms', **kwargs):
    """
    Generate texts from the input.

    Parameters
    ----------
    strings : List[str]
    to_lang: str, optional (default='ms')
        target language to translate.
    **kwargs: vector arguments pass to huggingface `generate` method.
        Read more at https://huggingface.co/docs/transformers/main_classes/text_generation

        If you are using `use_ctranslate2`, vector arguments pass to ctranslate2 `translate_batch` method.
        Read more at https://opennmt.net/CTranslate2/python/ctranslate2.Translator.html?highlight=translate_batch#ctranslate2.Translator.translate_batch

    Returns
    -------
    result: List[str]
    """
```

In [6]:
from pprint import pprint

In [7]:
# https://www.sinarharian.com.my/article/89678/BERITA/Politik/Saya-tidak-mahu-sentuh-isu-politik-Muhyiddin

string_news1 = 'TANGKAK - Tan Sri Muhyiddin Yassin berkata, beliau tidak mahu menyentuh mengenai isu politik buat masa ini, sebaliknya mahu menumpukan kepada soal kebajikan rakyat serta usaha merancakkan semula ekonomi negara yang terjejas berikutan pandemik Covid-19. Perdana Menteri menjelaskan perkara itu ketika berucap pada Majlis Bertemu Pemimpin bersama pemimpin masyarakat Dewan Undangan Negeri (DUN) Gambir di Dewan Serbaguna Bukit Gambir hari ini.'
pprint(string_news1)

('TANGKAK - Tan Sri Muhyiddin Yassin berkata, beliau tidak mahu menyentuh '
 'mengenai isu politik buat masa ini, sebaliknya mahu menumpukan kepada soal '
 'kebajikan rakyat serta usaha merancakkan semula ekonomi negara yang terjejas '
 'berikutan pandemik Covid-19. Perdana Menteri menjelaskan perkara itu ketika '
 'berucap pada Majlis Bertemu Pemimpin bersama pemimpin masyarakat Dewan '
 'Undangan Negeri (DUN) Gambir di Dewan Serbaguna Bukit Gambir hari ini.')


In [8]:
# https://www.sinarharian.com.my/article/90021/BERITA/Politik/Tun-Mahathir-Anwar-disaran-bersara-untuk-selesai-kemelut-politik

string_news2 = 'ALOR SETAR - Kemelut politik Pakatan Harapan (PH) belum berkesudahan apabila masih gagal memuktamadkan calon Perdana Menteri yang dipersetujui bersama. Ahli Parlimen Sik, Ahmad Tarmizi Sulaiman berkata, sehubungan itu pihaknya mencadangkan mantan Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu), Tun Dr Mahathir Mohamad dan Presiden Parti Keadilan Rakyat (PKR), Datuk Seri Anwar Ibrahim mengundurkan diri daripada politik sebagai jalan penyelesaian.'
pprint(string_news2)

('ALOR SETAR - Kemelut politik Pakatan Harapan (PH) belum berkesudahan apabila '
 'masih gagal memuktamadkan calon Perdana Menteri yang dipersetujui bersama. '
 'Ahli Parlimen Sik, Ahmad Tarmizi Sulaiman berkata, sehubungan itu pihaknya '
 'mencadangkan mantan Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu), Tun '
 'Dr Mahathir Mohamad dan Presiden Parti Keadilan Rakyat (PKR), Datuk Seri '
 'Anwar Ibrahim mengundurkan diri daripada politik sebagai jalan penyelesaian.')


In [9]:
string_news3 = 'Menteri Kanan (Kluster Keselamatan) Datuk Seri Ismail Sabri Yaakob berkata, kelonggaran itu diberi berikutan kerajaan menyedari masalah yang dihadapi mereka untuk memperbaharui dokumen itu. Katanya, selain itu, bagi rakyat asing yang pas lawatan sosial tamat semasa Perintah Kawalan Pergerakan (PKP) pula boleh ke pejabat Jabatan Imigresen yang terdekat untuk mendapatkan lanjutan tempoh.'
pprint(string_news3)

('Menteri Kanan (Kluster Keselamatan) Datuk Seri Ismail Sabri Yaakob berkata, '
 'kelonggaran itu diberi berikutan kerajaan menyedari masalah yang dihadapi '
 'mereka untuk memperbaharui dokumen itu. Katanya, selain itu, bagi rakyat '
 'asing yang pas lawatan sosial tamat semasa Perintah Kawalan Pergerakan (PKP) '
 'pula boleh ke pejabat Jabatan Imigresen yang terdekat untuk mendapatkan '
 'lanjutan tempoh.')


In [10]:
# https://qcikgubm.blogspot.com/2018/02/contoh-soalan-dan-jawapan-karangan.html

string_karangan = 'Selain itu, pameran kerjaya membantu para pelajar menentukan kerjaya yang akan diceburi oleh mereka. Seperti yang kita ketahui, pasaran kerjaya di Malaysia sangat luas dan masih banyak sektor pekerjaan di negara ini yang masih kosong kerana sukar untuk mencari tenaga kerja yang benar-benar berkelayakan. Sebagai contohnya, sektor perubatan di Malaysia menghadapi masalah kekurangan tenaga kerja yang kritikal, khususnya tenaga pakar disebabkan peletakan jawatan oleh doktor dan pakar perubatan untuk memasuki sektor swasta serta berkembangnya perkhidmatan kesihatan dan perubatan. Setelah menyedari  hakikat ini, para pelajar akan lebih berminat untuk menceburi bidang perubatan kerana pameran kerjaya yang dilaksanakan amat membantu memberikan pengetahuan am tentang kerjaya ini'
pprint(string_karangan)

('Selain itu, pameran kerjaya membantu para pelajar menentukan kerjaya yang '
 'akan diceburi oleh mereka. Seperti yang kita ketahui, pasaran kerjaya di '
 'Malaysia sangat luas dan masih banyak sektor pekerjaan di negara ini yang '
 'masih kosong kerana sukar untuk mencari tenaga kerja yang benar-benar '
 'berkelayakan. Sebagai contohnya, sektor perubatan di Malaysia menghadapi '
 'masalah kekurangan tenaga kerja yang kritikal, khususnya tenaga pakar '
 'disebabkan peletakan jawatan oleh doktor dan pakar perubatan untuk memasuki '
 'sektor swasta serta berkembangnya perkhidmatan kesihatan dan perubatan. '
 'Setelah menyedari  hakikat ini, para pelajar akan lebih berminat untuk '
 'menceburi bidang perubatan kerana pameran kerjaya yang dilaksanakan amat '
 'membantu memberikan pengetahuan am tentang kerjaya ini')


In [11]:
%%time

pprint(model.generate([string_news1, string_news2, string_news3, string_karangan], 
                      to_lang = 'en', max_length = 1000))

spaces_between_special_tokens is deprecated and will be removed in transformers v5. It was adding spaces between `added_tokens`, not special tokens, and does not exist in our fast implementation. Future tokenizers will handle the decoding process on a per-model rule.


['TANGKAK - Tan Sri Muhyiddin Yassin said he does not want to touch on '
 'political issues for now, but instead wants to focus on the welfare of the '
 "people and efforts to revitalize the country's economy which has been "
 'affected due to the Covid-19 pandemic. The Prime Minister explained the '
 'matter when speaking at the Meeting of Leaders with community leaders of the '
 'Gambir State Legislative Assembly at the Bukit Gambir Multipurpose Hall '
 'today.',
 'ALOR SETAR - The Pakatan Harapan (PH) political turmoil has not ended when '
 'it still failed to finalize the candidate for Prime Minister agreed upon. '
 'Sik Member of Parliament, Ahmad Tarmizi Sulaiman said, in that regard, he '
 'proposed that former Chairman of Parti Pribumi Bersatu Malaysia (Bersatu), '
 'Tun Dr Mahathir Mohamad and Parti Keadilan Rakyat (PKR) President, Datuk '
 'Seri Anwar Ibrahim to withdraw from politics as a solution.',
 'Senior Minister (Security Cluster) Datuk Seri Ismail Sabri Yaakob said th

### Able to translate code

In [12]:
s = """
`Untuk menggunakan numpy, pertama, Anda perlu menginstalnya melalui pip. Anda dapat melakukannya dengan menjalankan perintah `pip install numpy` di terminal Anda.
Setelah numpy terinstal, Anda dapat mengimpor modul numpy dengan menambahkan baris `import numpy as np` di awal program Anda.
Berikut adalah contoh beberapa operasi dasar numpy:
``` python
import numpy as np
# membuat array numpy dari list
my_list = [1, 2, 3, 4, 5]
my_array = np.array(my_list)
# membuat array numpy dengan rentang nilai tertentu
my_range = np.arange(0, 10, 2) # nilai awal, nilai akhir, dan loncatan
# membuat array numpy dengan nilai acak
my_random_array = np.random.rand(3, 3) # 3 baris dan 3 kolom
# mengakses elemen array numpy
print(my_array[0]) # mengakses elemen pertama
# melakukan operasi matematika pada array numpy
my_array = my_array + 1 # menambah setiap elemen dengan 1
my_array = my_array * 2 # mengalikan setiap elemen dengan 2
# mengubah bentuk array numpy
my_array = np.reshape(my_array, (2, 5)) # menjadi array 2D dengan 2 baris dan 5 kolom
```
Itulah beberapa operasi dasar numpy. Anda dapat menemukan dokumentasi resmi numpy di https://numpy.org/doc/stable/.
"""

print(model.generate([s], to_lang = 'ms', max_length = 1000)[0])

`Untuk menggunakan numpy, pertama, anda perlu memasangnya melalui pip. Anda boleh lakukannya dengan menjalankan perintah `pip install numpy` di terminal anda.
Selepas numpy dipasang, anda boleh import modul numpy dengan menambah baris `import numpy as np` pada permulaan program anda.
Berikut adalah contoh beberapa operasi asas numpy:
``` python
import numpy as np
# buat array numpy dari senarai
my_list = [1, 2, 3, 4, 5]
my_array = np.array(my_list)
# buat array numpy dengan julat nilai tertentu
my_range = np.arange(0, 10, 2) # nilai awal, nilai akhir, dan lompatan
# buat array numpy dengan nilai rawak
my_random_array = np.random.rand(3, 3) # 3 baris dan 3 lajur
# akses elemen array numpy
print(my_array[0]) # akses elemen pertama
# lakukan operasi matematik pada array numpy
my_array = my_array + 1 # tambah setiap elemen dengan 1
my_array = my_array * 2 # darab setiap elemen dengan 2
# ubah bentuk array numpy
my_array = np.reshape(my_array, (2, 5)) # menjadi array 2D dengan 2 baris dan 5

### Able to maintain input structure

In [13]:
s = """
\n26 DR.27.10.2021 \n\n \nUSUL MENANGGUHKAN MESYUARAT  \n\nDI BAWAH P.M. 18(1) \n \n\nPENJUALAN ASET GAS PETRONAS DI AZERBAIJAN \n\n \n\n11.33 pg. \n\nDato’ Seri Anwar bin Ibrahim [Port Dickson]: Tuan Yang di-Pertua, saya \nmohon mengemukakan usul di bawah Peraturan Mesyuarat 18(1) dan 18(2) Peraturan-\nperaturan Majlis Mesyuarat Dewan Rakyat seperti berikut: \n\n“Bahawa Dewan yang bersidang pada hari ini merundingkan \nlaporan bahawa Petronas telah menjual aset gas di Azerbaijan dengan \nharga hampir RM10 bilion berikutan desakan kerajaan menaikkan \nbayaran dividen Petronas sebanyak RM25 bilion kepada kerajaan bagi \ntahun 2021.” \n\n \nPerkara ini adalah perkara tertentu kerana penjualan aset gas di Azerbaijan \n\nbernilai RM10 bilion dan juga pembayaran dividen kepada kerajaan sebanyak RM25 bilion \nbagi tahun 2021 adalah berbahaya dan menjejaskan masa depan Petronas. Terkait \ndengan― sedikit penjelasan.  Kerana tindakan seperti ini telah menyebabkan J.P. Morgan \nmengeluarkan Petronas dari ESG Emerging Market Bond Index dan ESG Asia Credit \nIndex.  \n\nOleh itu, perkara itu perlu disegerakan kerana Petronas mengalami defisit \nberikutan pembayaran dividen berlebihan sejak tahun 2018 sehingga kini yang memaksa \nPetronas menjual aset atau meminjam lebih banyak dana luar negara bagi menampung \naliran tunai.   \n\nIni diperkukuh dengan kenyataan Tengku Muhammad Taufik, CEO Petronas, “If \nthere is a policy shift but it comes too suddenly or too slowly, not only Petronas but the \nentire Malaysian economy could be put at risk, with regards to our energy mix.” \n\nIni perkara berkenaan dengan kepentingan orang ramai, malah seluruh ekonomi \nnegara, kerana Petronas adalah institusi penting ekonomi negara dan sewajarnya \ndiperkukuh kekuatannya dan tidak dilemahkan penarafan syarikat. Kemampuan untuk \nmelabur bagi menjana keuntungan masa depan yang mampu untuk membayar dividen \nyang munasabah kepada kerajaan demi kepentingan rakyat. Dengan tambahan RM7 \nbilion tahun ini, hasil Petronas akan berjumlah RM44.8 bilion atau 18 peratus daripada \npendapatan kerajaan bagi tahun 2021 daripada RM37.8 bilion sebelumnya. \n\nJadi, mohon persetujuan Tuan Yang di-Pertua untuk dibahaskan. [Tepuk] \n\nTuan Yang di-Pertua: Terima kasih Yang Berhormat. Sebenarnya Yang \nBerhormat perlu melengkapkan teks Yang Berhormat dan cuma dibaca teks tersebut \nmengikut peraturan. Akan tetapi, tak apa.  \n\nAhli-ahli Yang Berhormat, saya telah menerima satu pemberitahu usul di bawah \nPeraturan Mesyuarat 18(1) oleh Yang Berhormat Port Dickson, Ketua Pembangkang pada \nhari Isnin, 25 Oktober 2021. Teks usul itu adalah seperti yang dibacakan tadi dengan \nsedikit tambahan oleh Ahli Yang Berhormat Port Dickson sebentar tadi.  \n\nBagi membolehkan perkara ini ditimbangkan oleh Majlis Mesyuarat, saya \nhendaklah berpuas hati bahawa perkara yang dibangkitkan oleh Yang Berhormat itu \nmematuhi tiga syarat seperti mana biasalah iaitu: \n\n(i) perkara tertentu; \n\n(ii) bagi kepentingan orang ramai; dan \n\n(iii) berkehendak disegerakan. \n\n\n
"""
print(model.generate([s], to_lang = 'en', max_length = 1000)[0])

26 DR. 27.10.2021

 
MOTION TO ADJOURN THE MEETING

UNDER P.M. 18(1)
 

PETRONAS GAS ASSET SALE IN AZERBAIJAN

 

11.33 a.m.

Dato' Seri Anwar bin Ibrahim [Port Dickson]: Mr. Speaker, I am
please submit a motion under Meeting Rules 18(1) and 18(2) Regulations-
 Rules of the Dewan Rakyat Meeting as follows:

"That the House in session today negotiated
report that Petronas has sold gas assets in Azerbaijan with
price almost RM10 billion following the government's insistence on raising
Petronas dividend payment of RM25 billion to the government for
year 2021.”

 
This is a certain matter because of the sale of gas assets in Azerbaijan

worth RM10 billion and also dividend payments to the government amounting to RM25 billion
for 2021 is dangerous and affects Petronas' future. Related
with a little explanation. Because actions like this have caused J.P. Morgan
issued Petronas from ESG Emerging Market Bond Index and ESG Asia Credit
Index.

Therefore, the matter needs to be expedited because 

### Able to translate indonesian, banjarese and javanese

In [14]:
s1 = 'Kita ora duwe wektu kanggo gelut utawa sengit.'
s2 = 'Tiga dekade kemudian area sekolah diperluas.'
s3 = 'Maklum haja lah muntung guri wadi kawa haja dipisiti, amun muntung manusia siapang nang hingkat manduga.'

In [15]:
pprint(model.generate([s1, s2, s3], to_lang = 'en', max_length = 1000))

["We don't have time to fight or be fierce.",
 'Three decades later the school area was expanded.',
 "It's understandable that the people are not ready to be in a bad mood, but "
 'the people are not ready to be in a bad mood.']


In [16]:
pprint(model.generate([s1, s2, s3], to_lang = 'ms', max_length = 1000))

['Kita tidak sempat untuk bertarung atau membenci.',
 'Tiga dekad kemudian kawasan sekolah telah diperluaskan.',
 '"Anda tahu bahawa anda hanya perlu memikirkan perkara yang tidak perlu, '
 'tetapi jika anda tidak mahu orang lain yang tidak tahu apa yang perlu '
 'dilakukan."']


### Able to translate Mandarin and local Mandarin

In [20]:
s1 = """
2023年了，依然还存在有很多封建的父母。来聊聊你所遇到的 奇奇怪怪的限制
以及 男与女不同对待 程度

已嫁女儿 不能回娘家吃团圆饭，不可以拜天公，不可以扫墓
红包 外孙 内孙 数目不一
可是 有事情，都是找女儿
"""
s2 = """
在選購外套時，防水功能成為了許多人關注的重點之一。極度乾燥外套以其卓越的防水性能和時尚設計，成為了眾多顧客的首選。讓我們來探討一下 極度乾燥 外套的防水技術，以及如何確保你在任何天氣下都能保持乾爽舒適。
Superdry 外套 防水高效的防水技術
"""

In [21]:
pprint(model.generate([s1, s2], to_lang = 'en', max_length = 1000))

["It has been 2023, and there are still many principled parents. Let's chat "
 'about the strange restrictions you encountered, and the level of treatment '
 'between men and women. If you marry your daughter, you cannot go back to the '
 "mother's house for a round trip, you cannot worship the gods, you cannot "
 'clean up the grave. The red envelope, the outside grandson, and the inner '
 'grandson, are not the same. However, when there is something, you can always '
 'look for your daughter.',
 'When choosing a pair of shoes, waterproofing is one of the focus of many '
 'people. The highly selesa air conditioning shoes are a popular choice for '
 "many customers. Let's explore the highly selesa waterproofing and "
 'waterproofing technology for the highly selesa waterproofing and safety of '
 'your shoes. Superdry is a high-quality waterproofing and waterproofing '
 'technology.']


In [22]:
pprint(model.generate([s1, s2], to_lang = 'ms', max_length = 1000))

['Sejak tahun 2023, masih banyak ibu bapa yang berdedikasi. Mari kita '
 'berbincang tentang batasan aneh yang anda hadapi dan tahap perlakuan yang '
 'berbeza antara lelaki dan perempuan.\n'
 '\n'
 'Sudah berkahwin dengan anak perempuan, tidak boleh pulang ke rumah ibu untuk '
 'makan jamuan, tidak boleh menyembah tuhan, tidak boleh menyapu kubur, dan '
 'tidak boleh menyapu kubur.\n'
 '\n'
 'Bungkusan merah, anak luar, anak luar, jumlah tidak sama.\n'
 '\n'
 'Tetapi jika ada sesuatu, semuanya akan mencari anak perempuan.',
 'Apabila memilih jaket, fungsi pertahanan air menjadi salah satu fokus utama '
 'ramai orang. Jaket yang sangat menyejukkan dengan prestasi dan penampilan '
 'yang cemerlang dalam pertahanan air, menjadi pilihan utama ramai pelanggan. '
 'Mari kita kunjungi teknologi pertahanan air yang sangat menyejukkan dan '
 'memastikan anda sentiasa selesa dan selesa di mana-mana cuaca. Jaket Super '
 'Dry teknologi pertahanan air yang cekap.']
